# Testing Notebook
The purpose of this notebook is to verify that all the components of the larger program are functioning properly.

# Setup
This section checks to see if all the libraries import correctly and that we can access the GPUs

In [1]:
# Imports
import torch
import numpy as np

from agents.dqn_agent import QNetwork, DQNAgent
from common.replay_buffer import ReplayBuffer, Experience

print("Imports successful!")

# Define configuration parameters for testing
# These would normally come from a .yaml file

# Environment/Wrapper params
INPUT_SHAPE = (4, 84, 84) # (num_stack, height, width)
NUM_ACTIONS = 9          # Ms. Pac-Man has 9 actions

# Replay Buffer params
BUFFER_CAPACITY = 1000
BATCH_SIZE = 4           # Use a small batch size for easy testing

# Set the device for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Imports successful!
Using device: cpu


# Testing QNetwork
The following checks to make sure that we can successfully create an instance of a QNetwork

In [2]:
print("--- Testing QNetwork ---")

# Instantiate the network
q_network = QNetwork(input_shape=INPUT_SHAPE, num_actions=NUM_ACTIONS).to(device)
print("Successfully instantiated QNetwork.")
print(q_network) # Print the network architecture

# Create a dummy input batch
dummy_input_shape = (BATCH_SIZE,) + INPUT_SHAPE 
# Create a numpy array of random data with this shape
dummy_np_array = np.random.rand(*dummy_input_shape).astype(np.uint8)

print(f"\nCreated a dummy numpy input with shape: {dummy_np_array.shape} and dtype: {dummy_np_array.dtype}")

# Prepare the data for the network
dummy_tensor = torch.tensor(dummy_np_array, dtype=torch.float32).to(device) / 255.0

print(f"Converted to a dummy torch tensor with shape: {dummy_tensor.shape} and dtype: {dummy_tensor.dtype}")

# Perform a forward pass
with torch.no_grad(): # Disable gradient calculations because we are not training
    output = q_network(dummy_tensor)

print(f"\nPerformed forward pass successfully.")
print(f"Output tensor shape: {output.shape}")
print(f"Expected output shape: {(BATCH_SIZE, NUM_ACTIONS)}")

# Final check
if output.shape == (BATCH_SIZE, NUM_ACTIONS):
    print("\nTest PASSED: Output shape is correct.")
else:
    print(f"\nTest FAILED: Output shape is {output.shape}, but expected {(BATCH_SIZE, NUM_ACTIONS)}.")

--- Testing QNetwork ---
Successfully instantiated QNetwork.
QNetwork(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=9, bias=True)
  )
)

Created a dummy numpy input with shape: (4, 4, 84, 84) and dtype: uint8
Converted to a dummy torch tensor with shape: torch.Size([4, 4, 84, 84]) and dtype: torch.float32

Performed forward pass successfully.
Output tensor shape: torch.Size([4, 9])
Expected output shape: (4, 9)

Test PASSED: Output shape is correct.


# Test replay_buffer.py

In [3]:
print('\n--- Testing Replay Buffer ---')

# Instantiate the replay buffer
replay_buffer = ReplayBuffer(capacity=BUFFER_CAPACITY, batch_size=BATCH_SIZE)
print(f"Successfully instantiated ReplayBuffer with capacity: {BUFFER_CAPACITY} and batch size: {BATCH_SIZE}")

# Create dummy experiences and add them to the buffer

num_experiences_to_add = 10
print(f"\nAdding {num_experiences_to_add} dummy experiences to the replay buffer...")

for i in range(num_experiences_to_add):
    dummy_state = np.ones(INPUT_SHAPE) * i
    dummy_action = i % NUM_ACTIONS
    dummy_reward = float(i)
    dummy_next_state = np.ones(INPUT_SHAPE) * (i + 1)
    dummy_done = (i == num_experiences_to_add - 1) # Last experience is terminal

    replay_buffer.add(dummy_state, dummy_action, dummy_reward, dummy_next_state, dummy_done)

# Check length of buffer
print(f"Current buffer size: {len(replay_buffer)} (should be {num_experiences_to_add})")
if len(replay_buffer) == num_experiences_to_add:
    print("TEST PASSED: Replay buffer size is correct.")
else:
    print("TEST FAILED: Replay buffer size is incorrect.")

# Sample a batch
print(f"\nSampling a batch of size {BATCH_SIZE} from the replay buffer...")
states, actions, rewards, next_states, dones = replay_buffer.sample()

print(f"Sampled states shape: {states.shape}, Expected: ({BATCH_SIZE}, {INPUT_SHAPE})")
print(f"Sampled actions shape: {actions.shape}, Expected: ({BATCH_SIZE},)")
print(f"Sampled rewards shape: {rewards.shape}, Expected: ({BATCH_SIZE},)")
print(f"Sampled next_states shape: {next_states.shape}, Expected: ({BATCH_SIZE}, {INPUT_SHAPE})")
print(f"Sampled dones shape: {dones.shape}, Expected: ({BATCH_SIZE},)")

# Final check for sampled batch shapes
correct_shapes = all([
    states.shape == (BATCH_SIZE, *INPUT_SHAPE),
    actions.shape == (BATCH_SIZE,),
    rewards.shape == (BATCH_SIZE,),
    next_states.shape == (BATCH_SIZE, *INPUT_SHAPE),
    dones.shape == (BATCH_SIZE,)
])

if correct_shapes:
    print("TEST PASSED: Sampled batch shapes are correct.")
else:
    print("TEST FAILED: Sampled batch shapes are incorrect.")



--- Testing Replay Buffer ---
Successfully instantiated ReplayBuffer with capacity: 1000 and batch size: 4

Adding 10 dummy experiences to the replay buffer...
Current buffer size: 10 (should be 10)
TEST PASSED: Replay buffer size is correct.

Sampling a batch of size 4 from the replay buffer...
Sampled states shape: (4, 4, 84, 84), Expected: (4, (4, 84, 84))
Sampled actions shape: (4,), Expected: (4,)
Sampled rewards shape: (4,), Expected: (4,)
Sampled next_states shape: (4, 4, 84, 84), Expected: (4, (4, 84, 84))
Sampled dones shape: (4,), Expected: (4,)
TEST PASSED: Sampled batch shapes are correct.


# Testing DQN Agent

In [ ]:
print("\n--- Testing DQNAgent ---")

# 1. Instantiate the agent
# We'll use the config parameters from Cell 1
agent = DQNAgent(
    input_shape=INPUT_SHAPE,
    num_actions=NUM_ACTIONS,
    replay_buffer_capacity=BUFFER_CAPACITY,
    batch_size=BATCH_SIZE,
    learning_rate=0.001, # A dummy learning rate
    gamma=0.99,
    device=device
)
print("Successfully instantiated DQNAgent.")

# 2. Test the `act` method
print("\n--- Testing act() method ---")
# Create a dummy state
dummy_state = np.random.rand(*INPUT_SHAPE)
# Test with high epsilon (should be random)
action_explore = agent.act(dummy_state, epsilon=1.0)
print(f"Action taken with epsilon=1.0 (explore): {action_explore}")
# Test with zero epsilon (should be exploit)
action_exploit = agent.act(dummy_state, epsilon=0.0)
print(f"Action taken with epsilon=0.0 (exploit): {action_exploit}")

if isinstance(action_explore, int) and isinstance(action_exploit, int):
    print("Test PASSED: act() method returns an integer action.")
else:
    print("Test FAILED: act() method did not return an integer.")


# 3. Test the `learn` method
print("\n--- Testing learn() method ---")
# First, populate the buffer with enough experiences to form a batch
print(f"Populating replay buffer with {BATCH_SIZE} experiences...")
for _ in range(BATCH_SIZE):
    dummy_experience = (
        np.random.rand(*INPUT_SHAPE), 
        np.random.randint(NUM_ACTIONS), 
        np.random.rand(), 
        np.random.rand(*INPUT_SHAPE), 
        False
    )
    agent.replay_buffer.add(*dummy_experience)

# Now, call the learn method
try:
    loss = agent.learn()
    print(f"Successfully called learn() method. Loss: {loss}")
    if isinstance(loss, float):
        print("Test PASSED: learn() method ran without errors and returned a float.")
    else:
        print("Test FAILED: learn() method did not return a float.")
except Exception as e:
    print(f"Test FAILED: learn() method raised an exception: {e}")


# 4. Test the target network update
print("\n--- Testing target network update ---")
try:
    agent.update_target_network()
    # Check if weights are indeed the same (optional, but good)
    policy_dict = agent.q_policy_net.state_dict()
    target_dict = agent.q_target_net.state_dict()
    is_same = all(torch.equal(policy_dict[key], target_dict[key]) for key in policy_dict)
    
    if is_same:
        print("Test PASSED: Target network weights match policy network weights after update.")
    else:
        print("Test FAILED: Target network weights do not match.")

except Exception as e:
    print(f"Test FAILED: update_target_network() raised an exception: {e}")


--- Testing DQNAgent ---
Successfully instantiated DQNAgent.

--- Testing act() method ---
Action taken with epsilon=1.0 (explore): 6
Action taken with epsilon=0.0 (exploit): 2
✅ Test PASSED: act() method returns an integer action.

--- Testing learn() method ---
Populating replay buffer with 4 experiences...
Successfully called learn() method. Loss: 0.5260008573532104
✅ Test PASSED: learn() method ran without errors and returned a float.

--- Testing target network update ---
✅ Test PASSED: Target network weights match policy network weights after update.
